In [1]:
import sqlite3

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
plt.style.use('seaborn')

# 2021-09-25.db

In [4]:
connection = sqlite3.connect('/home/javier/Desktop/TFM/Fraud_Detection_In_The_Irish_Rental_Market/data/2021-09-25.db') #### change
cursor = connection.cursor()

In [5]:
query = '''SELECT * FROM rent;'''

In [6]:
daft_09_25 = pd.read_sql_query(query, connection)
daft_09_25.shape

(3092, 19)

In [7]:
connection.close()

In [8]:
rent_1 = daft_09_25.copy()

In [9]:
print(rent_1.shape)
rent_1.drop(['contact', 'phone'], axis=1, inplace=True)
print(rent_1.shape)

(3092, 19)
(3092, 17)


# 2021-09-26.db

In [10]:
connection = sqlite3.connect('/home/javier/Desktop/TFM/Fraud_Detection_In_The_Irish_Rental_Market/data/2021-09-26.db')
cursor = connection.cursor()

In [11]:
daft_09_26 = pd.read_sql_query(query, connection)
daft_09_26.shape

(3071, 19)

In [12]:
connection.close()

In [13]:
rent_2 = daft_09_26.copy()

In [14]:
print(rent_2.shape)
rent_2.drop(['contact', 'phone'], axis=1, inplace=True)
print(rent_2.shape)

(3071, 19)
(3071, 17)


--------------

In [15]:
er_1 = rent_1['entered_renewed'].value_counts().head(15).sort_index(ascending=False)
er_1

24/09/2021    1245
23/09/2021     506
22/09/2021     303
21/09/2021     238
20/09/2021     185
19/09/2021      97
18/09/2021      91
17/09/2021      92
16/09/2021      54
15/09/2021      51
14/09/2021      47
13/09/2021      64
12/09/2021       9
11/09/2021      13
10/09/2021      22
Name: entered_renewed, dtype: int64

In [16]:
er_2 = rent_2['entered_renewed'].value_counts().head(15).sort_index(ascending=False)
er_2

25/09/2021    576
24/09/2021    922
23/09/2021    432
22/09/2021    272
21/09/2021    209
20/09/2021    158
19/09/2021     84
18/09/2021     78
17/09/2021     38
16/09/2021     48
15/09/2021     49
14/09/2021     48
13/09/2021     61
11/09/2021     13
09/09/2021      8
Name: entered_renewed, dtype: int64

In [17]:
combined = pd.merge(left=er_1, right=er_2, how='right', 
                    left_index=True, right_index=True, 
                    suffixes=('_09-25','_09-26'))
combined

,entered_renewed_09-25,entered_renewed_09-26
25/09/2021,NaN,576
24/09/2021,1245.0,922
23/09/2021,506.0,432
22/09/2021,303.0,272
21/09/2021,238.0,209
20/09/2021,185.0,158
19/09/2021,97.0,84
18/09/2021,91.0,78
17/09/2021,92.0,38
16/09/2021,54.0,48


In [18]:
combined['dif'] = combined['entered_renewed_09-26'] - combined['entered_renewed_09-25']
combined

,entered_renewed_09-25,entered_renewed_09-26,dif
25/09/2021,NaN,576,NaN
24/09/2021,1245.0,922,-323.0
23/09/2021,506.0,432,-74.0
22/09/2021,303.0,272,-31.0
21/09/2021,238.0,209,-29.0
20/09/2021,185.0,158,-27.0
19/09/2021,97.0,84,-13.0
18/09/2021,91.0,78,-13.0
17/09/2021,92.0,38,-54.0
16/09/2021,54.0,48,-6.0


El 09/14 aparece un nuevo anuncio en los datos referentes al dia 09/26, lo cual no tiene sentido.

In [19]:
rent_1[rent_1['entered_renewed'] == '14/09/2021'].shape

(47, 17)

In [20]:
rent_2[rent_2['entered_renewed'] == '14/09/2021'].shape

(48, 17)

In [24]:
for url_2 in rent_2.loc[rent_2['entered_renewed'] == '14/09/2021', 'url']:
    if url_2 not in list(rent_1.loc[rent_1['entered_renewed'] == '14/09/2021', 'url']):
        print(url_2)

https://www.daft.ie/for-rent/house-greenfield-park-lahinch-co-clare/3525333


In [26]:
rent_2.loc[rent_2['url'] == 'https://www.daft.ie/for-rent/house-greenfield-park-lahinch-co-clare/3525333']

,daft_id,item_id,url,name,price,info,overview,facilities,ber,entered_renewed,views,psr,type_house,energy_performance_indicator,coordinates,type,scraping_date
2975,26122691,3525333,https://www.daft.ie/for-rent/house-greenfield-...,"Greenfield Park, Lahinch, Co. Clare",€800 per month,"3 Bed,3 Bath,House","Double Bedroom: 3,Bathroom: 3,Available From: ...","Parking,Central Heating,Cable Television,Washi...",SI_666,14/09/2021,"4,141",none,house,none,52.93434860495016+-9.334020947657336,rent,2021-9-26


In [27]:
rent_1.loc[rent_1['url'] == 'https://www.daft.ie/for-rent/house-greenfield-park-lahinch-co-clare/3525333']

,daft_id,item_id,url,name,price,info,overview,facilities,ber,entered_renewed,views,psr,type_house,energy_performance_indicator,coordinates,type,scraping_date


------------

In [28]:
def get_db(date, query = '''SELECT * FROM rent;'''): # 2021-09-26
    
    connection = sqlite3.connect('/home/javier/Desktop/TFM/Fraud_Detection_In_The_Irish_Rental_Market/data/{}.db')
    cursor = connection.cursor()
    
    daft = pd.read_sql_query(query, connection)
    connection.close()
    rent = daft.copy()
    rent.drop(['contact', 'phone'], axis=1, inplace=True)
    
    return rent

In [29]:
!pwd

/home/javier/Desktop/TFM/Fraud_Detection_In_The_Irish_Rental_Market


In [30]:
!ls /home/javier/Desktop/TFM/Fraud_Detection_In_The_Irish_Rental_Market/data

2021-09-22_3_websites.db  2021-09-27.db  2021-09-30.db	2021-10-03.db
2021-09-25.db		  2021-09-28.db  2021-10-01.db	backup
2021-09-26.db		  2021-09-29.db  2021-10-02.db	share.csv


In [31]:
!ls data

2021-09-22_3_websites.db  2021-09-27.db  2021-09-30.db	2021-10-03.db
2021-09-25.db		  2021-09-28.db  2021-10-01.db	backup
2021-09-26.db		  2021-09-29.db  2021-10-02.db	share.csv


In [32]:
from os import listdir
from os.path import isfile, join

In [33]:
data_path = 'data'

listdir(data_path)

['2021-09-29.db',
 'share.csv',
 '2021-10-01.db',
 '2021-09-28.db',
 '2021-09-27.db',
 '2021-10-03.db',
 '2021-09-30.db',
 '2021-10-02.db',
 '2021-09-26.db',
 'backup',
 '2021-09-22_3_websites.db',
 '2021-09-25.db']

In [34]:
[f for f in listdir(data_path) if isfile(join(data_path, f))]

['2021-09-29.db',
 'share.csv',
 '2021-10-01.db',
 '2021-09-28.db',
 '2021-09-27.db',
 '2021-10-03.db',
 '2021-09-30.db',
 '2021-10-02.db',
 '2021-09-26.db',
 '2021-09-22_3_websites.db',
 '2021-09-25.db']

In [35]:
isfile(join(data_path, listdir(data_path)[0]))

True

In [36]:
import re

In [38]:
date_pattern = r'\d{4}-\d{2}-\d{2}.db'

print(re.search(date_pattern, listdir(data_path)[0]))

<re.Match object; span=(0, 13), match='2021-09-29.db'>


In [39]:
date_pattern = r'\d{4}-\d{2}-\d{2}.db'

re.match(date_pattern, listdir(data_path)[0])

<re.Match object; span=(0, 13), match='2021-09-29.db'>

In [40]:
date_pattern = r'\d{4}-\d{2}-\d{2}.db'

if re.match(date_pattern, listdir(data_path)[0]):
    print(listdir(data_path)[0]) # and isfile(join(data_path, f))

2021-09-29.db


In [68]:
import re
from os import listdir
from os.path import isfile, join

data_pattern = r'\d{4}-\d{2}-\d{2}.db'
daily_db = [f for f in listdir(data_path) if isfile(join(data_path, f)) \
            and re.match(date_pattern, f)]
daily_db

['2021-09-29.db',
 '2021-10-01.db',
 '2021-09-28.db',
 '2021-09-27.db',
 '2021-10-03.db',
 '2021-09-30.db',
 '2021-10-02.db',
 '2021-09-26.db',
 '2021-09-25.db']

-----------------

In [69]:
def get_db(dbname, query='''SELECT * FROM rent;'''): # 2021-09-26
    
    connection = sqlite3.connect(f'/home/javier/Desktop/TFM/Fraud_Detection_In_The_Irish_Rental_Market/data/{dbname}')
    cursor = connection.cursor()
    
    daft = pd.read_sql_query(query, connection)
    connection.close()
    daft.drop(['contact', 'phone'], axis=1, inplace=True)
    
    rent = daft.copy() 
    
    return rent

In [8]:
import re
from os import listdir
from os.path import isfile, join


def get_db(dbname, query='''SELECT * FROM rent;'''): # 2021-09-26
    
    connection = sqlite3.connect(f'/home/javier/Desktop/TFM/Fraud_Detection_In_The_Irish_Rental_Market/data/{dbname}')
    cursor = connection.cursor()
    
    daft = pd.read_sql_query(query, connection)
    connection.close()
    daft.drop(['contact', 'phone'], axis=1, inplace=True)
    
    rent = daft.copy() 
    
    return rent


def db_dict():
    
    data_path = 'data'
    #data_pattern = r'\d{4}-\d{2}-\d{2}.db'
    daily_db = [f for f in listdir(data_path) if isfile(join(data_path, f)) \
            and re.match(r'\d{4}-\d{2}-\d{2}.db', f)]
    #daily_db
    
    db_dict = {}
    for dbname in daily_db:
        
        db = get_db(dbname)
        key = dbname.split('.')[0]    
        if key in db_dict:
            pass
        else:
            db_dict[key] = db
            
    return db_dict 

In [10]:
test_db_dict = db_dict()
test_db_dict.keys()

dict_keys(['2021-09-29', '2021-10-04', '2021-10-01', '2021-09-28', '2021-09-27', '2021-10-03', '2021-09-30', '2021-10-02', '2021-09-26', '2021-09-25'])

In [11]:
for key in test_db_dict.keys():
    print(key)

2021-09-29
2021-10-04
2021-10-01
2021-09-28
2021-09-27
2021-10-03
2021-09-30
2021-10-02
2021-09-26
2021-09-25


In [14]:
sorted(test_db_dict.keys()) #ordenarlas mas adelante

['2021-09-25',
 '2021-09-26',
 '2021-09-27',
 '2021-09-28',
 '2021-09-29',
 '2021-09-30',
 '2021-10-01',
 '2021-10-02',
 '2021-10-03',
 '2021-10-04']

Ahora voy a empezar a analizar uno de los datasets enteros y el mes de septiembre. Usare el 2021-10-01 para tomar los datos.